# Closures

Closure in Python is an inner function object, a function that behaves like an object, that remembers and has access to variables in the local scope in which it was created even after the outer function has finished executing. 


vanaf nu zullen we in plaats van de inner() functie aan te roepen in de outer() functie, de inner functie teruggeven.

In [ ]:
def outer():
    a = 10

    def inner(x):
        print(a + x)
    return inner

func = outer()
func(2)

Wat gebeurd er ?

We binden de outer functie aan de variabele func door `func = outer()`. 

outer() returned inner => inner is dus aan func gebonden.

Als we daarna "()" toevoegen aan func zal de functie worden uitgevoerd.

## multi-scoped variables  en Python cell

We herhalen hieronder de code van daarnet.

In [ ]:
def outer():
    a = 10

    def inner():
        print(a)
    return inner

func = outer()
func()

In deze code zien we dat `a` voorkomt in 2 verschillende scopes. We noemen `a` een "multi-scoped" variabele. We noemen dit een "free variabele".

Python bewaart deze 'gedeelde' variabele in de inner-scope. Hierdoor kunnen we deze nog gebruiken zelfs nadat de outer functie is uitgevoerd.

Python maakt een tussentijds object "cell" waarbij het memory address wordt bewaart van de variabele.

De scopes bevatten een referentie van variabele `a` naar deze cell.

In dit voorbeeld wordt zowel binnen de inner als outer scope een referentie naar dezelfde cell en memory address bijgehouden. Deze cell heeft op zijn beurt een referentie naar een "integer object" dat een memory address en een waarde (hier => 10) heeft.
Eenmaal deze cell aangemaakt is, zelfs nadat de outer functie is uitgevoerd, zal "cell" beschikbaar blijven.

In [ ]:
def outer():
    a = "10"
    x = 3

    def inner():
        x = 5
        print(a)
    return inner

func = outer()

In [ ]:
func.__closure__

In [ ]:
func.__code__.co_freevars

In [1]:
def outer():
    a = "10"
    x = 3
    z = 5
    
    def inner():
        x = 5
        print(a, z)
    return inner

func = outer()

print(func.__closure__)
print(func.__code__.co_freevars)

(<cell at 0x000001E78302DB70: str object at 0x000001E7FF243B70>, <cell at 0x000001E78302D720: int object at 0x000001E7FE340170>)
('a', 'z')


Wanneer een closure wordt aangemaakt, dan is er een referentie via de cell naar de free-variable.

Ook al is dit een indirecte referentie, het memory address (id()) is dit van de "value" zelf.

In [2]:
def outer():
    a = 10
    print(id(a))
    
    def inner():
        print(id(a))
        print(a)
    return inner

func = outer()
func()

2095913894416
2095913894416
10


### multiple instances of the closures && shared extended scopes

In [4]:
def outer():
    c = 0
    
    def counter():
        nonlocal c
        c += 1
        return c
    return counter

func_1 = outer()
func_1()

1

In [10]:
func_1.__closure__

(<cell at 0x000001E7842C8340: int object at 0x000001E7FE340130>,)

In [11]:
func_1()

4

In [5]:
func_1.__code__.co_freevars

('c',)

In [12]:
func_2 = outer()
print(func_2())
print(func_2())

1
2


Wanneer we opnieuw de outer functie oproepen (func_2) starten we opnieuw van 0. 

Elke keer dat we de outer functie aanroepen maken we een nieuwe scope (nieuwe closure).

Omdat deze scopes niet gedeeld worden, starten ze dus telkens opnieuw.

Als we naar hun "cell" kijken, zien we dat deze verschillend zijn.

We zien echter wel dat ze naar hetzelfde "integer object" verwijzen voor hun "free variabele"

In [13]:
print(func_1.__closure__)
print(func_2.__closure__)

(<cell at 0x000001E7842C8340: int object at 0x000001E7FE340150>,)
(<cell at 0x000001E7842C9810: int object at 0x000001E7FE340110>,)


In [15]:
print(func_2())
print(func_1.__closure__)
print(func_2.__closure__)

4
(<cell at 0x000001E7842C8340: int object at 0x000001E7FE340150>,)
(<cell at 0x000001E7842C9810: int object at 0x000001E7FE340150>,)


Hier zien we dus dat func_2 naar een 'integer object' verwijst met een ander address.

Dit is omdat we func_2 nogmaals hebben gerund. 

In [16]:
def outer():
    c = 0
    
    def counter_1():
        nonlocal c
        c += 1
        return c
    def counter_2():
        nonlocal c
        c += 1
        return c
    
    return counter_1, counter_2

func_1, func_2 = outer()

In [17]:
print(func_1.__closure__)
print(func_2.__closure__)

(<cell at 0x000001E78300DDB0: int object at 0x000001E7FE3400D0>,)
(<cell at 0x000001E78300DDB0: int object at 0x000001E7FE3400D0>,)


In [18]:
print(func_1())
print(func_1())
print(func_2())
print(func_2())

1
2
3
4


In [19]:
print(func_1.__closure__)
print(func_2.__closure__)

(<cell at 0x000001E78300DDB0: int object at 0x000001E7FE340150>,)
(<cell at 0x000001E78300DDB0: int object at 0x000001E7FE340150>,)


Hier zien we dus dat func_1 en func_2 nog steeds naar dezelfde cell verwijzen en dus ook naar hetzelfde "integer object" address blijven verwijzen.

Ook al hebben we func_1 al 2 kere gerund en func_2 nog niet.

## Free variable as an argument

In [20]:
def outer(n):
    def inner(x):
        return n + x
    return inner

func_1 = outer(1)
print(func_1(2))
print(func_1.__code__.co_freevars)

3
('n',)


In [21]:
func_2 = outer(2)
func_3 = outer(3)

In [22]:
print(func_1(5))
print(func_2(5))
print(func_3(5))

6
7
8


Als we meerder instanties willen aanmaken? 

Kunnen we dat niet met een for loop en lambda? 

In [23]:
ls = []

for i in range(1, 9):
    ls.append(lambda x: x + i)

In [25]:
print(ls)

[<function <lambda> at 0x000001E782F9B370>, <function <lambda> at 0x000001E784A6E050>, <function <lambda> at 0x000001E784A6D7E0>, <function <lambda> at 0x000001E784A6E680>, <function <lambda> at 0x000001E784A6E3B0>, <function <lambda> at 0x000001E784A6D6C0>, <function <lambda> at 0x000001E784A6E5F0>, <function <lambda> at 0x000001E784A6E710>]


onze lambda is zelf een closure. Omdat het een vrije variabele `i` heeft.

In [26]:
print(ls[0](5))
print(ls[1](5))
print(ls[7](5))

13
13
13


Wat gebeurd er ?

De lambda functie heeft een vrije variabele, namelijk `i`.

deze refereert naar de het object "loop variabele". Bij elke interatie veranderd deze.

omdat elke closure naar het zelfde object verwijst. deze krijgt de waarde "8" nadat alle iteraties doorlopen zijn.

Als we daarna dus 5 meegeven, krijgen we 13 voor alle closures.

In [29]:
for i in range(len(ls)):
    print(i,'-' , ls[i](5))
    

0 - 5
1 - 6
2 - 7
3 - 8
4 - 9
5 - 10
6 - 11
7 - 12


In [ ]:
def outer():
    def inner():
        print(1)
    return inner

func = outer()

print(func())
print(func.__code__.co_freevars)
print(func.__closure__)

In [31]:
def outer():
    a = 2
    def inner():
        print(1)
    return inner

func = outer()

print(func())
print("------------------")
print(func.__code__.co_freevars)
print(func.__closure__)

1
None
------------------
()
None


In [34]:
def outer():
    a = 2
    def inner():
        print(a)
    return inner

func = outer()

print(func())
print(func.__code__.co_freevars)
print(func.__closure__)

2
None
('a',)
(<cell at 0x000001E784253850: int object at 0x000001E7FE340110>,)


## Nested closures

wanneer we nested scopes hebben. En elk van deze scopes hebben free variable, than hebben we nested closures.

In [35]:
def outer(n):
    def inner_1(x):
        current = x
        def inner_2():
            nonlocal current
            current += n
            return current
        return inner_2
    return inner_1

func_1 = outer(10)  # => func_1 = inner_1 


In [36]:
print(func_1.__code__.co_freevars)

('n',)


In [37]:
func_2 = func_1(7)
print(func_2.__code__.co_freevars)

('current', 'n')


In [39]:
func_2()

27